In [2]:
import pp
import numpy as np
from pyPoseidon.dem import *
from pyPoseidon.utils.fix import *
import shapely

## Set

In [3]:
dic = {'minlon': 0., # lat/lon window
       'maxlon' : 13.,
       'minlat' : 58.,
       'maxlat' : 66.5,
       'dpath':'/Users/brey/DATA/GEBCO_2014_2D.nc'}

In [4]:
## lat,lon grid
resolution=.1
lon=np.arange(dic['minlon'],dic['maxlon'],resolution)
lat=np.arange(dic['minlat'],dic['maxlat'],resolution)
x, y = np.meshgrid(lon,lat)

In [5]:
dic.update({'grid_x':x, 'grid_y':y})

In [6]:
#Define the coastline shapefile
shapefile = '/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_%sm_coastline.shp' % \
                    {'l':110, 'i':50, 'h':10}['i']

In [7]:
dem = gebco(**dic) # get bathymetry


extracting dem from /Users/brey/DATA/GEBCO_2014_2D.nc

.. interpolating on grid ..

dem done


In [8]:
#define coastline
    
shp = gp.GeoDataFrame.from_file(shapefile)

           
xp = dem.Dataset.ilons.values
yp = dem.Dataset.ilats.values
    
    
#put them all in a list
ls=[]
for i in range(shp.shape[0]):
    il = shp.loc[i,'geometry']
    try:
        #print len(il)
        for k in range(len(list(il.geoms))): # for MultiLineStrings
            ls.append(list(il.geoms)[k])
    except:
        ls.append(il)
            
sall = geometry.MultiLineString(ls) # join
c = ops.linemerge(sall) #merge
    
#Select the Line Strings that correspond to our grid
#create a polygon of the grid
grp=geometry.Polygon([(xp.min(),yp.min()),(xp.min(),yp.max()),(xp.max(),yp.max()),(xp.max(),yp.min())])    
    
cl=[] #initialize
#add Polygons if they intersect with the grid
for i in range(len(c)):
    z = geometry.Polygon(c[i])
    if z.intersects(grp): 
        cl.append(c[i])
    
cll = geometry.MultiLineString(cl) #join them into a Multiline
cg = ops.linemerge(cll) #merge parts if possible
    


In [25]:
def loop0(xp,yp,cg):
#check if the grid polygons intersect the shoreline
    gps = []
    igps = []
    for i in range(0,xp.shape[0]-1):
        for j in range(0,xp.shape[1]-1):
            p = shapely.geometry.Polygon([(xp[i,j],yp[i,j]),(xp[i+1,j],yp[i+1,j]),(xp[i+1,j+1],yp[i+1,j+1]),(xp[i,j+1],yp[i,j+1])])
            if not p.intersects(cg): 
                gps.append(p)
                igps.append([i+1,j+1])
                if i == 0 : 
                    igps.append([i,j])
                    igps.append([i,j+1])
                if j == 0 : 
                    igps.append([i,j])
                    igps.append([i+1,j])
    return igps

## One core

In [26]:
#%%timeit 
g1 = loop0(xp,yp,cg)

In [27]:
solo = np.array(g1)

In [28]:
solo.shape

(10545, 2)

## PP loop

In [9]:
def loop(xp,yp,cg):
#check if the grid polygons intersect the shoreline
    gps = []
    igps = []
    for i in range(xp.shape[0]-1):
        for j in range(xp.shape[1]-1):
            p = shapely.geometry.Polygon([(xp[i,j],yp[i,j]),(xp[i+1,j],yp[i+1,j]),(xp[i+1,j+1],yp[i+1,j+1]),(xp[i,j+1],yp[i,j+1])])
            if not p.intersects(cg): 
                gps.append(p)
                igps.append([i+1,j+1])
    
    return igps

In [10]:
job_server = pp.Server() 

INFO:pp:Creating server instance (pp-1.6.6)
INFO:pp:Running on Python 2.7.15 darwin
INFO:pp:pp local server started with 8 workers


In [11]:
job_server.get_ncpus()

8

In [12]:
ncpus=4

In [13]:
job_server.set_ncpus(ncpus)

In [14]:
xstep=xp.shape[0]/ncpus
ystep=xp.shape[1]/ncpus
xstep,ystep

(21, 32)

In [15]:
xp.shape

(85, 130)

In [17]:
#%%timeit
jobs=[]
for l in range(ncpus):
#    xl=xp[l*xstep:(l+1)*xstep,l*ystep:(l+1)*ystep]
#    yl=yp[l*xstep:(l+1)*xstep,l*ystep:(l+1)*ystep]   
    xl=xp[l*xstep:(l+1)*xstep+1,:]
    yl=yp[l*xstep:(l+1)*xstep+1,:]   


    jobs.append(job_server.submit(loop,(xl,yl,cg,),modules=('shapely',)))

ps=jobs[0]()
for l in range(1,ncpus):    
    ps = np.vstack([ps, [[i+l*xstep,j] for [i,j] in jobs[l]()]])

INFO:pp:Task 8 started
INFO:pp:Task 9 started
INFO:pp:Task 10 started
INFO:pp:Task 11 started


In [29]:
ps.shape

(10153, 2)

## Add extra points from boundary


In [32]:
bgps=[]
for [i,j] in ps:
    if i == 1 : 
        bgps.append([i-1,j-1])
        bgps.append([i-1,j])
    if j == 1 : 
        bgps.append([i-1,j-1])
        bgps.append([i,j-1])

In [34]:
len(bgps) + ps.shape[0]

10545

In [35]:
ps = ps = np.vstack([ps, bgps]) # final stack

## Compare

In [50]:
ps.sort(axis=0)

In [52]:
ps

array([[  0,   0],
       [  0,   0],
       [  0,   1],
       ...,
       [ 84, 129],
       [ 84, 129],
       [ 84, 129]])

In [53]:
solo.sort(axis=0)

In [54]:
solo

array([[  0,   0],
       [  0,   0],
       [  0,   1],
       ...,
       [ 84, 129],
       [ 84, 129],
       [ 84, 129]])

In [55]:
np.array_equal(ps,solo)

True